# Single Fully-Connected Network (TensorFlow)

<a href="https://colab.research.google.com/github/VertaAI/modeldb-client/blob/development/workflows/demos/tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# restart your notebook if prompted on Colab
try:
    import verta
except ModuleNotFoundError:
    !pip install verta

This example features:
- **Tensorflow**'s Keras API for building neural networks
- **verta**'s Python client

In [ ]:
HOST = "demo2.verta.ai"
PORT = "6244"

PROJECT_NAME = "MNIST Multiclassification"
EXPERIMENT_NAME = "FC-NN"

## Imports

In [ ]:
import time

import numpy as np
import matplotlib.pyplot as plt

from sklearn import datasets

import tensorflow as tf
from tensorflow import keras

---

# Log Workflow

## Prepare Data

In [ ]:
data = datasets.load_digits()

X = data['data']
y = data['target']

## Prepare Hyperparameters

In [ ]:
hyperparams = {
    'hidden_size': 512,
    'dropout': 0.2,
    'batch_size': 1024,
    'num_epochs': 10,
    'optimizer': "adam",
    'loss': "sparse_categorical_crossentropy",
    'validation_split': 0.1,
}

## Instantiate Client

In [ ]:
from verta import ModelDBClient

client = ModelDBClient(HOST, PORT)
proj = client.set_project(PROJECT_NAME)
expt = client.set_experiment(EXPERIMENT_NAME)

## Create Keras Callback

In [ ]:
run = client.set_experiment_run()

## Define and Log Model

In [ ]:
run.log_hyperparameters(**hyperparams)

model = keras.models.Sequential()
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(hyperparams['hidden_size'], activation=tf.nn.relu))
model.add(keras.layers.Dropout(rate=hyperparams['dropout']))
model.add(keras.layers.Dense(10, activation=tf.nn.softmax))

model.compile(optimizer=hyperparams['optimizer'],
              loss=hyperparams['loss'],
              metrics=['accuracy'])

## Run and Log Validation

In [ ]:
def log_validation_callback(epoch, logs):  # Keras will call this each epoch
    run.log_observation("train_loss", float(logs['loss']))
    run.log_observation("train_acc", float(logs['acc']))
    run.log_observation("val_loss", float(logs['val_loss']))
    run.log_observation("val_acc", float(logs['val_acc']))

model.fit(X, y,
          validation_split=hyperparams['validation_split'],
          batch_size=hyperparams['batch_size'], epochs=hyperparams['num_epochs'],
          callbacks=[keras.callbacks.LambdaCallback(on_epoch_end=log_validation_callback)])

train_loss, train_acc = model.evaluate(X, y)
run.log_metric("train_loss", train_loss)
run.log_metric("train_acc", train_acc)